In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
from scipy.optimize import leastsq
from numpy import exp
from scipy import ndimage
from func import *

In [2]:
def scale_factor_spectrum(z,data,error,wo_wave):
    k = 1+z
    select1 = (wo_wave>6750*k) & (wo_wave<6800*k)
    cont_dat = data[select1]
    cont_err = error[select1]
    error_maincube = np.std(cont_dat)
    error_errcube = np.mean(cont_err)
    fact = error_maincube/error_errcube
    return fact

In [3]:
def create_3_arcsec_minicube(wo_cube,wo_err,wo_header,brightest_pixel_x,brightest_pixel_y,box_size):
    mini_cube_data = wo_cube[:,brightest_pixel_y-box_size:brightest_pixel_y+box_size+1,brightest_pixel_x-box_size:brightest_pixel_x+box_size+1]
    mini_cube_err = wo_err[:,brightest_pixel_y-box_size:brightest_pixel_y+box_size+1,brightest_pixel_x-box_size:brightest_pixel_x+box_size+1]
    wo_header['CRPIX1'] = wo_header['CRPIX1'] - (brightest_pixel_x-box_size)
    wo_header['CRPIX2'] = wo_header['CRPIX2'] - (brightest_pixel_y-box_size)
    return mini_cube_data, mini_cube_err,wo_header

In [4]:
def scale_factor_spectrum(z,data,error,wo_wave):
    k = 1+z
    select1 = (wo_wave>6750*k) & (wo_wave<6850*k)
    cont_dat = data[select1]
    cont_err = error[select1]
    error_maincube = np.std(cont_dat)
    error_errcube = np.mean(cont_err)
    fact = error_maincube/error_errcube
    return fact

def emp_fact(z,mini_cube,mini_err,wo_wave):
    true_err = np.zeros((np.shape(wo_wave)[0],mini_cube.shape[1],mini_cube.shape[2]),dtype=np.float32)
    for i in range(mini_cube.shape[1]):
        for j in range(mini_cube.shape[2]):
            k = 1+z
            select1 = (wo_wave>6750*k) & (wo_wave<6850*k)
   
            (spectrum,error) = (mini_cube[:,i,j],mini_err[:,i,j])  
            (spec,err) = (spectrum[select1],error[select1])
            (error_maincube,error_errcube) = (np.std(spec),np.mean(err))
            fact = error_maincube/error_errcube
            new_err = fact*error
            true_err[:,i,j] = new_err 
    return true_err
    

In [7]:
def algorithmic_script(obj,z,box_size=60,min_wave=4730,max_wave=5120,destination_path_cube="/media/mainak/Seagate/MUSE NFM"):
    (wo_cube,wo_err,wo_wave,wo_header) = loadCube('%s/%s/%s_NFM.final.fits'%(destination_path_cube,obj,obj)) 
    #(cont_cube,cont_err,cont_wave,cont_header) = loadCube('%s/MUSE/%s/fitting/full/%s.cont_model.fits'%(prefix_path_cube,obj,obj))
    #[x0,y0] = [166,165]#agn_location(obj)
    [brightest_pixel_x,brightest_pixel_y] = [166,165]
    sampling_rate = 0.025#sampling_size(cont_cube)
    
    (mini_cube,mini_err,wo_header)=create_3_arcsec_minicube(wo_cube,wo_err,wo_header,brightest_pixel_x,brightest_pixel_y,box_size=60)
    true_err = emp_fact(z,mini_cube,mini_err,wo_wave)
    k = 1+z
    select = (wo_wave > min_wave*k) & (wo_wave < max_wave*k)  
    mini_header = wo_header
    mini_wave = wo_wave[select]
    mini_cube_Hb = mini_cube[select]
    mini_err = true_err[select]
    
    store_cube('%s/%s/%s.1_arcsec_minicube.fits'%(destination_path_cube,obj,obj),mini_cube_Hb,mini_wave,mini_err,mini_header)
    print ('%s'%(obj))

In [8]:
z = {"HE1029-1401":0.08582}

objs = z.keys()


for obj in objs:
    algorithmic_script(obj,z[obj])              


HE1029-1401


In [10]:
z = {"HE0021-1810":0.05352,"HE0021-1819":0.053197,"HE0040-1105":0.041692,"HE0108-4743":0.02392,"HE0114-0015":0.04560
    ,"HE0119-0118":0.054341,"HE0212-0059":0.026385,"HE0224-2834":0.059800,"HE0227-0913":0.016451,"HE0232-0900":0.043143
    ,"HE0253-1641":0.031588,"HE0345+0056":0.031,"HE0351+0240":0.036,"HE0412-0803":0.038160,"HE0429-0247":0.042009
    ,"HE0433-1028":0.035550,"HE0853+0102":0.052,"HE0934+0119":0.050338,"HE1011-0403":0.058314,"HE1017-0305":0.049986
    ,"HE1029-1831":0.040261,"HE1107-0813":0.058,"HE1108-2813":0.024013,"HE1126-0407":0.061960,"HE1237-0504":0.009
    ,"HE1248-1356":0.01465,"HE1330-1013":0.022145,"HE1353-1917":0.035021,"HE1417-0909":0.044,"HE2128-0221":0.05248
    ,"HE2211-3903":0.039714,"HE2222-0026":0.059114,"HE2233+0124":0.056482,"HE2302-0857":0.046860}

objs = z.keys()


for obj in objs:
    algorithmic_script(obj,z[obj])              


HE0021-1810
HE0021-1819
HE0040-1105
HE0108-4743
HE0114-0015
HE0119-0118
HE0212-0059
HE0224-2834
HE0227-0913
HE0232-0900
HE0253-1641
HE0345+0056
HE0351+0240
HE0412-0803
HE0429-0247
HE0433-1028
HE0853+0102
HE0934+0119
HE1011-0403
HE1017-0305
HE1029-1831
HE1107-0813
HE1108-2813
HE1126-0407
HE1237-0504
HE1248-1356
HE1330-1013
HE1353-1917
HE1417-0909
HE2128-0221
HE2211-3903
HE2222-0026
HE2233+0124
HE2302-0857
